In [2]:
# Define the LEMCell
class LEMCell(nn.Module):
    def __init__(self, ninp, nhid, dt):
        super(LEMCell, self).__init__()
        self.ninp = ninp
        self.nhid = nhid
        self.dt = dt
        self.inp2hid = nn.Linear(ninp, 4 * nhid)
        self.hid2hid = nn.Linear(nhid, 3 * nhid)
        self.transform_z = nn.Linear(nhid, nhid)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / np.sqrt(self.nhid)
        for w in self.parameters():
            w.data.uniform_(-std, std)

    def forward(self, x, y, z):
        transformed_inp = self.inp2hid(x)
        transformed_hid = self.hid2hid(y)
        i_dt1, i_dt2, i_z, i_y = transformed_inp.chunk(4, 1)
        h_dt1, h_dt2, h_y = transformed_hid.chunk(3, 1)

        ms_dt_bar = self.dt * torch.sigmoid(i_dt1 + h_dt1)
        ms_dt = self.dt * torch.sigmoid(i_dt2 + h_dt2)

        z = (1. - ms_dt) * z + ms_dt * torch.tanh(i_y + h_y)
        y = (1. - ms_dt_bar) * y + ms_dt_bar * torch.tanh(self.transform_z(z) + i_z)

        return y, z

# Define the LEM model
class LEM(nn.Module):
    def __init__(self, ninp, nhid, nout, dt=1.):
        super(LEM, self).__init__()
        self.nhid = nhid
        self.cell = LEMCell(ninp, nhid, dt)
        self.classifier = nn.Linear(nhid, nout)
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if 'classifier' in name and 'weight' in name:
                nn.init.kaiming_normal_(param.data)

    def forward(self, input):
        y = input.data.new(input.size(1), self.nhid).zero_()
        z = input.data.new(input.size(1), self.nhid).zero_()
        for x in input:
            y, z = self.cell(x, y, z)
        out = self.classifier(y)
        return out


### PINN data importing

In [3]:
# importing data

# Load the .mat file
mat_data = scipy.io.loadmat('burg.mat')

# Access the variables stored in the .mat file
# The variable names in the .mat file become keys in the loaded dictionary
x = mat_data['x']
t = mat_data['t']
u = mat_data['u1']



### Exact Solution data importing

In [4]:
# importing data

import torch
import torch.nn as nn
import numpy as np
import time
import scipy.io

# Load the .mat file
mat_data = scipy.io.loadmat('burgers_shock.mat')

# Access the variables stored in the .mat file
# The variable names in the .mat file become keys in the loaded dictionary
x = mat_data['x']
t = mat_data['t']
u_1 = mat_data['usol']


In [5]:
u_1.shape

(256, 100)

In [6]:
# new dataset generation with errors
u_1 = torch.tensor(u_1)
u = torch.tensor(u) + 0.5
k1 = ( u - u_1)**2
u_test_full_tensor = torch.tensor(u_1**2)
u_test_full_tensor.shape

/tmp/ipykernel_11699/1797348989.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  u_test_full_tensor = torch.tensor(u_1**2)


torch.Size([256, 100])

In [7]:
# Compute the relative L2 error norm (generalization error)
relative_error_test = torch.mean(k1) / torch.mean(u_test_full_tensor)

# Format the relative error to display 10 digits of magnitude
formatted_relative_error_test = "{:.50f}".format(relative_error_test.item())

print("Relative Error Test: ", formatted_relative_error_test, "%")


Relative Error Test:  0.66240856126090141842155389895197004079818725585938 %


In [8]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Toy problem data
input_size = 256
hidden_size = 32
output_size = 256
sequence_length = 79
batch_size = 1
num_epochs = 20000

# Set random seed for reproducibility
torch.manual_seed(42)
u[:, 0:100].shape

torch.Size([256, 100])

In [9]:
input_data = u[:,0:79]
target_data = u[:,1:80]

test_data = u[:,79]
#test_target = u[:,80:100]

print("test data shape", test_data.shape)
#print("test target shape", test_target.shape)

print("input data shape",input_data.shape)
print("Target data shape",target_data.shape)

# Convert data to tensors
input_tensor = torch.tensor(input_data.T).view(batch_size, sequence_length, input_size).float()
target_tensor = torch.tensor(target_data.T).view(batch_size, sequence_length, output_size).float()

print("input tensor shape",input_tensor.shape)
print("Target tensor shape",target_tensor.shape)

test data shape torch.Size([256])
input data shape torch.Size([256, 79])
Target data shape torch.Size([256, 79])
input tensor shape torch.Size([1, 79, 256])
Target tensor shape torch.Size([1, 79, 256])


/tmp/ipykernel_11699/3906878071.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(input_data.T).view(batch_size, sequence_length, input_size).float()
/tmp/ipykernel_11699/3906878071.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_tensor = torch.tensor(target_data.T).view(batch_size, sequence_length, output_size).float()


In [10]:
# Convert test data to tensors
test_tensor = torch.tensor(test_data.T).view(batch_size, 1, input_size).float()
#test_target_tensor = torch.tensor(test_target.T).view(batch_size, 20, output_size).float()
target_tensor = torch.squeeze(target_tensor)

/tmp/ipykernel_11699/546962278.py:2: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484746322/work/aten/src/ATen/native/TensorShape.cpp:2981.)
  test_tensor = torch.tensor(test_data.T).view(batch_size, 1, input_size).float()
/tmp/ipykernel_11699/546962278.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_tensor = torch.tensor(test_data.T).view(batch_size, 1, input_size).float()


In [11]:
# Create LEM instance
lem = LEM(input_size, hidden_size, output_size, dt=0.1)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(lem.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    output = lem(input_tensor)
    loss = criterion(output, target_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.16f}')

Epoch: 10/20000, Loss: 0.6200895905494690
Epoch: 20/20000, Loss: 0.5730171799659729
Epoch: 30/20000, Loss: 0.5228640437126160
Epoch: 40/20000, Loss: 0.4732925891876221
Epoch: 50/20000, Loss: 0.4271131157875061
Epoch: 60/20000, Loss: 0.3852492868900299
Epoch: 70/20000, Loss: 0.3494604229927063
Epoch: 80/20000, Loss: 0.3178244531154633
Epoch: 90/20000, Loss: 0.2895337343215942
Epoch: 100/20000, Loss: 0.2640845775604248
Epoch: 110/20000, Loss: 0.2410973161458969
Epoch: 120/20000, Loss: 0.2202756404876709
Epoch: 130/20000, Loss: 0.2013816386461258
Epoch: 140/20000, Loss: 0.1842183172702789
Epoch: 150/20000, Loss: 0.1686185002326965
Epoch: 160/20000, Loss: 0.1544376611709595
Epoch: 170/20000, Loss: 0.1415487378835678
Epoch: 180/20000, Loss: 0.1298387646675110
Epoch: 190/20000, Loss: 0.1192062497138977
Epoch: 200/20000, Loss: 0.1095592901110649
Epoch: 210/20000, Loss: 0.1008141413331032
Epoch: 220/20000, Loss: 0.0928942859172821
Epoch: 230/20000, Loss: 0.0857293680310249
Epoch: 240/20000, Lo

Epoch: 2100/20000, Loss: 0.0017563538858667
Epoch: 2110/20000, Loss: 0.0017236637650058
Epoch: 2120/20000, Loss: 0.0016926768003032
Epoch: 2130/20000, Loss: 0.0016609588637948
Epoch: 2140/20000, Loss: 0.0016305827302858
Epoch: 2150/20000, Loss: 0.0016010531689972
Epoch: 2160/20000, Loss: 0.0015722139505669
Epoch: 2170/20000, Loss: 0.0015440201386809
Epoch: 2180/20000, Loss: 0.0015164719661698
Epoch: 2190/20000, Loss: 0.0014895525528118
Epoch: 2200/20000, Loss: 0.0014632423408329
Epoch: 2210/20000, Loss: 0.0014375277096406
Epoch: 2220/20000, Loss: 0.0014123944565654
Epoch: 2230/20000, Loss: 0.0013878301251680
Epoch: 2240/20000, Loss: 0.0013638219097629
Epoch: 2250/20000, Loss: 0.0013403579359874
Epoch: 2260/20000, Loss: 0.0013174265623093
Epoch: 2270/20000, Loss: 0.0012950164964423
Epoch: 2280/20000, Loss: 0.0012731159804389
Epoch: 2290/20000, Loss: 0.0012517139548436
Epoch: 2300/20000, Loss: 0.0012307993602008
Epoch: 2310/20000, Loss: 0.0012103611370549
Epoch: 2320/20000, Loss: 0.00119

Epoch: 4200/20000, Loss: 0.0001557479554322
Epoch: 4210/20000, Loss: 0.0001543390244478
Epoch: 4220/20000, Loss: 0.0001529419969302
Epoch: 4230/20000, Loss: 0.0001515608892078
Epoch: 4240/20000, Loss: 0.0001506721891928
Epoch: 4250/20000, Loss: 0.0001488526322646
Epoch: 4260/20000, Loss: 0.0001480113132857
Epoch: 4270/20000, Loss: 0.0001463874214096
Epoch: 4280/20000, Loss: 0.0001448240509490
Epoch: 4290/20000, Loss: 0.0001435165322619
Epoch: 4300/20000, Loss: 0.0001422013592673
Epoch: 4310/20000, Loss: 0.0001409016986145
Epoch: 4320/20000, Loss: 0.0001396164007019
Epoch: 4330/20000, Loss: 0.0001383446942782
Epoch: 4340/20000, Loss: 0.0001370829995722
Epoch: 4350/20000, Loss: 0.0001358320296276
Epoch: 4360/20000, Loss: 0.0001345916680293
Epoch: 4370/20000, Loss: 0.0001333618565695
Epoch: 4380/20000, Loss: 0.0001321425370406
Epoch: 4390/20000, Loss: 0.0001309335348196
Epoch: 4400/20000, Loss: 0.0001297358976444
Epoch: 4410/20000, Loss: 0.0001286527985940
Epoch: 4420/20000, Loss: 0.00013

Epoch: 6140/20000, Loss: 0.0000403884769185
Epoch: 6150/20000, Loss: 0.0000306238980556
Epoch: 6160/20000, Loss: 0.0000301838936139
Epoch: 6170/20000, Loss: 0.0000298122249660
Epoch: 6180/20000, Loss: 0.0000295783574984
Epoch: 6190/20000, Loss: 0.0000293891371257
Epoch: 6200/20000, Loss: 0.0000291920168820
Epoch: 6210/20000, Loss: 0.0000289878225885
Epoch: 6220/20000, Loss: 0.0000287873099296
Epoch: 6230/20000, Loss: 0.0000285937985609
Epoch: 6240/20000, Loss: 0.0000284016077785
Epoch: 6250/20000, Loss: 0.0000282110104308
Epoch: 6260/20000, Loss: 0.0000280218937405
Epoch: 6270/20000, Loss: 0.0000278342249658
Epoch: 6280/20000, Loss: 0.0000276478695014
Epoch: 6290/20000, Loss: 0.0000274628491752
Epoch: 6300/20000, Loss: 0.0000272791330644
Epoch: 6310/20000, Loss: 0.0000270967411780
Epoch: 6320/20000, Loss: 0.0000269156207651
Epoch: 6330/20000, Loss: 0.0000267357918347
Epoch: 6340/20000, Loss: 0.0000265572198259
Epoch: 6350/20000, Loss: 0.0000263799029199
Epoch: 6360/20000, Loss: 0.00002

Epoch: 8130/20000, Loss: 0.0000092752743512
Epoch: 8140/20000, Loss: 0.0000091152169261
Epoch: 8150/20000, Loss: 0.0000090201856437
Epoch: 8160/20000, Loss: 0.0000089459817900
Epoch: 8170/20000, Loss: 0.0000088847809820
Epoch: 8180/20000, Loss: 0.0000088377701104
Epoch: 8190/20000, Loss: 0.0000087942717073
Epoch: 8200/20000, Loss: 0.0000087496928245
Epoch: 8210/20000, Loss: 0.0000087064900072
Epoch: 8220/20000, Loss: 0.0000086634499894
Epoch: 8230/20000, Loss: 0.0000086208474386
Epoch: 8240/20000, Loss: 0.0000085785486590
Epoch: 8250/20000, Loss: 0.0000085365663836
Epoch: 8260/20000, Loss: 0.0000084948551375
Epoch: 8270/20000, Loss: 0.0000084534503912
Epoch: 8280/20000, Loss: 0.0000084123248598
Epoch: 8290/20000, Loss: 0.0000083714785433
Epoch: 8300/20000, Loss: 0.0000083309150796
Epoch: 8310/20000, Loss: 0.0000082906062744
Epoch: 8320/20000, Loss: 0.0000082505875980
Epoch: 8330/20000, Loss: 0.0000082109609139
Epoch: 8340/20000, Loss: 0.0000081855750977
Epoch: 8350/20000, Loss: 0.00001

...

Epoch: 12080/20000, Loss: 0.0000016673061509
Epoch: 12090/20000, Loss: 0.0000014498327801
Epoch: 12100/20000, Loss: 0.0000013209836425
Epoch: 12110/20000, Loss: 0.0000012830339529
Epoch: 12120/20000, Loss: 0.0000012780637917
Epoch: 12130/20000, Loss: 0.0000012690828726
Epoch: 12140/20000, Loss: 0.0000012594273358
Epoch: 12150/20000, Loss: 0.0000012529264950
Epoch: 12160/20000, Loss: 0.0000012458050378
Epoch: 12170/20000, Loss: 0.0000012391938071
Epoch: 12180/20000, Loss: 0.0000012325581338
Epoch: 12190/20000, Loss: 0.0000012259976074
Epoch: 12200/20000, Loss: 0.0000012194919918
Epoch: 12210/20000, Loss: 0.0000012130292362
Epoch: 12220/20000, Loss: 0.0000012066033150
Epoch: 12230/20000, Loss: 0.0000012002160474
Epoch: 12240/20000, Loss: 0.0000011938642501
Epoch: 12250/20000, Loss: 0.0000011875455357
Epoch: 12260/20000, Loss: 0.0000011812628600
Epoch: 12270/20000, Loss: 0.0000011750149724
Epoch: 12280/20000, Loss: 0.0000011688208588
Epoch: 12290/20000, Loss: 0.0000011639813238
Epoch: 123

Epoch: 14030/20000, Loss: 0.0000005236981906
Epoch: 14040/20000, Loss: 0.0000005214757266
Epoch: 14050/20000, Loss: 0.0000005192665640
Epoch: 14060/20000, Loss: 0.0000005171888233
Epoch: 14070/20000, Loss: 0.0000005284485383
Epoch: 14080/20000, Loss: 0.0000032450504932
Epoch: 14090/20000, Loss: 0.0000045044321269
Epoch: 14100/20000, Loss: 0.0000012985217381
Epoch: 14110/20000, Loss: 0.0000007334536463
Epoch: 14120/20000, Loss: 0.0000005607990943
Epoch: 14130/20000, Loss: 0.0000005261146612
Epoch: 14140/20000, Loss: 0.0000005147980460
Epoch: 14150/20000, Loss: 0.0000005105007403
Epoch: 14160/20000, Loss: 0.0000005055844099
Epoch: 14170/20000, Loss: 0.0000004997537530
Epoch: 14180/20000, Loss: 0.0000004969562610
Epoch: 14190/20000, Loss: 0.0000004946768399
Epoch: 14200/20000, Loss: 0.0000004925038866
Epoch: 14210/20000, Loss: 0.0000004904553066
Epoch: 14220/20000, Loss: 0.0000004884659575
Epoch: 14230/20000, Loss: 0.0000004864805305
Epoch: 14240/20000, Loss: 0.0000004845204558
Epoch: 142

Epoch: 15960/20000, Loss: 0.0000003196527700
Epoch: 15970/20000, Loss: 0.0000003189289259
Epoch: 15980/20000, Loss: 0.0000003182045702
Epoch: 15990/20000, Loss: 0.0000003175628080
Epoch: 16000/20000, Loss: 0.0000003169477054
Epoch: 16010/20000, Loss: 0.0000003163478652
Epoch: 16020/20000, Loss: 0.0000003157540789
Epoch: 16030/20000, Loss: 0.0000003151677674
Epoch: 16040/20000, Loss: 0.0000003145879646
Epoch: 16050/20000, Loss: 0.0000003140125671
Epoch: 16060/20000, Loss: 0.0000003134441897
Epoch: 16070/20000, Loss: 0.0000003128894832
Epoch: 16080/20000, Loss: 0.0000003129722757
Epoch: 16090/20000, Loss: 0.0000003903466848
Epoch: 16100/20000, Loss: 0.0000108908607217
Epoch: 16110/20000, Loss: 0.0000011481171214
Epoch: 16120/20000, Loss: 0.0000008200233310
Epoch: 16130/20000, Loss: 0.0000004785063084
Epoch: 16140/20000, Loss: 0.0000003801980313
Epoch: 16150/20000, Loss: 0.0000003500188370
Epoch: 16160/20000, Loss: 0.0000003337695489
Epoch: 16170/20000, Loss: 0.0000003201786001
Epoch: 161

Epoch: 17870/20000, Loss: 0.0000002578678107
Epoch: 17880/20000, Loss: 0.0000005203983164
Epoch: 17890/20000, Loss: 0.0000090546382125
Epoch: 17900/20000, Loss: 0.0000007310044339
Epoch: 17910/20000, Loss: 0.0000005418845603
Epoch: 17920/20000, Loss: 0.0000004709115444
Epoch: 17930/20000, Loss: 0.0000003452717294
Epoch: 17940/20000, Loss: 0.0000002841460969
Epoch: 17950/20000, Loss: 0.0000002598477238
Epoch: 17960/20000, Loss: 0.0000002579581917
Epoch: 17970/20000, Loss: 0.0000002574032578
Epoch: 17980/20000, Loss: 0.0000002558047925
Epoch: 17990/20000, Loss: 0.0000002553722140
Epoch: 18000/20000, Loss: 0.0000002549034832
Epoch: 18010/20000, Loss: 0.0000002544891800
Epoch: 18020/20000, Loss: 0.0000002541283664
Epoch: 18030/20000, Loss: 0.0000002537759087
Epoch: 18040/20000, Loss: 0.0000002534326882
Epoch: 18050/20000, Loss: 0.0000002530966867
Epoch: 18060/20000, Loss: 0.0000002527663412
Epoch: 18070/20000, Loss: 0.0000002524384399
Epoch: 18080/20000, Loss: 0.0000002521144324
Epoch: 180

Epoch: 19840/20000, Loss: 0.0000002607196450
Epoch: 19850/20000, Loss: 0.0000003868623537
Epoch: 19860/20000, Loss: 0.0000002945962478
Epoch: 19870/20000, Loss: 0.0000002201454521
Epoch: 19880/20000, Loss: 0.0000002287185481
Epoch: 19890/20000, Loss: 0.0000002171015865
Epoch: 19900/20000, Loss: 0.0000002179158969
Epoch: 19910/20000, Loss: 0.0000002163352946
Epoch: 19920/20000, Loss: 0.0000002156631638
Epoch: 19930/20000, Loss: 0.0000002153125536
Epoch: 19940/20000, Loss: 0.0000002149830607
Epoch: 19950/20000, Loss: 0.0000002146635865
Epoch: 19960/20000, Loss: 0.0000002143618616
Epoch: 19970/20000, Loss: 0.0000002140700275
Epoch: 19980/20000, Loss: 0.0000002137820019
Epoch: 19990/20000, Loss: 0.0000002134995327
Epoch: 20000/20000, Loss: 0.0000002132230890


In [12]:
print(test_tensor.shape)
prediction_tensor = torch.zeros(1, 20, 256).float()
print(prediction_tensor.shape)

torch.Size([1, 1, 256])
torch.Size([1, 20, 256])


In [13]:
with torch.no_grad():
    prediction = lem(test_tensor)
    prediction = prediction.view(1, 1, 256).float()
    prediction_tensor[:, 0, :] = prediction
    for i in range(19):
        prediction = lem(prediction)
        prediction = prediction.view(1, 1, 256).float()
        prediction_tensor[:, i+1, :] = prediction
        

### Four different types of error

In [14]:
# Exact Solution

u_test = u_1.T
u_test_full = u_test[80:100, :]

In [15]:
prediction_tensor = torch.squeeze(prediction_tensor)


In [16]:
# Extrapolation

k1 = ( prediction_tensor - u_test_full)**2
u_test_full_tensor = torch.tensor(u_test_full**2)
u_test_full_tensor.shape

/tmp/ipykernel_11699/1744225238.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  u_test_full_tensor = torch.tensor(u_test_full**2)


torch.Size([20, 256])

### L^2 norm error

In [17]:
# Compute the relative L2 error norm (generalization error)
relative_error_test = torch.mean(k1)/ torch.mean(u_test_full_tensor)

print("Relative Error Test: ", relative_error_test.item(), "%")

Relative Error Test:  1.187628627069187 %


### Max absolute norm error

In [ ]:
R_abs = torch.max(torch.abs(prediction_tensor - u_test_full))

In [ ]:
print(R_abs)

### Explained variance score

In [ ]:
import torch

a = prediction_tensor
b = u_test_full
# Assuming 'a' is your predicted values (model's predictions) and 'b' is the true values (ground truth)
# Make sure 'a' and 'b' are PyTorch tensors
# a = torch.tensor(a)
b = torch.tensor(b)
# Calculate the mean of 'b'
mean_b = torch.mean(b)

# Calculate the Explained Variance Score
numerator = torch.var(b - a)  # Variance of the differences between 'b' and 'a'
denominator = torch.var(b)    # Variance of 'b'
evs = 1 - numerator / denominator

print("Explained Variance Score:", evs.item())

### Mean absolute error

In [ ]:
# Compute the relative L2 error norm (generalization error)
relative_error_test = torch.mean(torch.abs(prediction_tensor - u_test_full))

print("Relative Error Test: ", relative_error_test, "%")

### Contour plot for PINN (80 percent) and (20 percentage lem prediction)

In [ ]:
print(prediction_tensor.shape)
prediction_tensor = torch.squeeze(prediction_tensor)
input_tensor = torch.squeeze(input_tensor)

conc_u = torch.squeeze(input_tensor)
concatenated_tensor = torch.cat((conc_u, prediction_tensor), dim=0)

x1 = np.linspace(-1, 1, 256)
t1 = np.linspace(0, 1, 99)

### Snapshot time plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Create the figure and axis objects with reduced width
fig, ax = plt.subplots(figsize=(5, 5))  # You can adjust the width (7 inches) and height (5 inches) as needed




final_time_output = prediction_tensor[3, :]
final_out = final_time_output.detach().numpy().reshape(-1, 1)
final_true = u_1[:, 83].reshape(-1, 1)

# Plot the data with red and blue lines, one with dotted and one with solid style
ax.plot(x, final_out, color='red', linestyle='dotted', linewidth=12, label='Prediction')
ax.plot(x, final_true, color='blue', linestyle='solid', linewidth=7, label='True')


# Set the axis labels with bold font weight
ax.set_xlabel(r"${x}$", fontsize=26, color='black', fontdict={'weight': 'bold'})
ax.set_ylabel(r"${u(x, t)}$", fontsize=26, color='black', fontdict={'weight': 'bold'})

# Set the title with bold font weight
ax.set_title(r"${t = 0.83}$", fontsize=26, color='black', fontweight='bold')

# Set the number of ticks for x-axis and y-axis to 3
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

# Set tick labels fontweight to bold and increase font size
ax.tick_params(axis='both', which='major', labelsize=20, width=2, length=10)

# # Set the fontweight for tick labels to bold
# for tick in ax.get_xticklabels() + ax.get_yticklabels():
#     tick.set_weight('bold')

# Set the spines linewidth to bold
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)

# Set the legend
# ax.legend()

plt.savefig('LEM_0.83_20.pdf', dpi=500, bbox_inches="tight")
#plt.savefig('lem_0.83_20.png', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Create the figure and axis objects with reduced width
fig, ax = plt.subplots(figsize=(5, 5))  # You can adjust the width (7 inches) and height (5 inches) as needed



final_time_output = prediction_tensor[-2, :]
final_out = final_time_output.detach().numpy().reshape(-1, 1)
final_true = u_1[:, -2].reshape(-1, 1)

# Plot the data with red and blue lines, one with dotted and one with solid style
ax.plot(x, final_out, color='red', linestyle='dotted', linewidth=12, label='Prediction')
ax.plot(x, final_true, color='blue', linestyle='solid', linewidth=7, label='True')


# Set the axis labels with bold font weight
ax.set_xlabel(r"${x}$", fontsize=26, color='black', fontdict={'weight': 'bold'})
ax.set_ylabel(r"${u(x, t)}$", fontsize=26, color='black', fontdict={'weight': 'bold'})

# Set the title with bold font weight
ax.set_title(r"${t = 0.98}$", fontsize=26, color='black', fontweight='bold')

# Set the number of ticks for x-axis and y-axis to 3
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

# Set tick labels fontweight to bold and increase font size
ax.tick_params(axis='both', which='major', labelsize=20, width=2, length=10)

# # Set the fontweight for tick labels to bold
# for tick in ax.get_xticklabels() + ax.get_yticklabels():
#     tick.set_weight('bold')

# Set the spines linewidth to bold
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)

# Set the legend
# ax.legend()

plt.savefig('LEM_0.98_20.pdf', dpi=500, bbox_inches="tight")
#plt.savefig('lem_0.98_20.png', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


### Contour plot where 80 percent for PINN solution and 20 percent for lem solution

### Exact contour

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator

# Assuming you have defined concatenated_tensor as a PyTorch tensor
# concatenated_tensor = torch.cat((tensor1, tensor2), dim=0)

# Convert concatenated_tensor to a NumPy array
concatenated_array = u_1.T

# Define custom color levels
x = np.linspace(-1, 1, concatenated_array.shape[1])  # Replace 0 and 1 with your actual x range
t = np.linspace(0, 1, concatenated_array.shape[0])  # Replace 0 and 1 with your actual t range
X, T = np.meshgrid(x, t)

# Define custom color levels using the minimum and maximum from the NumPy array
c_levels = np.linspace(np.min(concatenated_array), np.max(concatenated_array), 400)

# Plot the contour with interpolated data
plt.figure(figsize=(20, 5))
plt.pcolormesh(T, X, concatenated_array, shading='auto', cmap='coolwarm')

# Set the fontweight for axis labels to regular (not bold)
plt.xlabel("$t$", fontsize=26)
plt.ylabel("$x$", fontsize=26)
plt.title("$u(x, t)$", fontsize=26)

# Set tick labels fontweight to regular (not bold) and increase font size
plt.tick_params(axis='both', which='major', labelsize=20, width=3, length=10)

# Set the fontweight for tick labels to regular (not bold)
for tick in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
    tick.set_weight('normal')

# Set the number of ticks for x-axis and y-axis to 5
num_ticks = 5
x_ticks = np.linspace(np.min(T), np.max(T), num_ticks)
y_ticks = np.linspace(np.min(X), np.max(X), num_ticks)

plt.gca().xaxis.set_major_locator(FixedLocator(x_ticks))
plt.gca().yaxis.set_major_locator(FixedLocator(y_ticks))

cbar1 = plt.colorbar()
# Set the number of ticks for the color bar with uniformly distributed numbers
num_ticks = 5
c_ticks = np.linspace(np.min(concatenated_array), np.max(concatenated_array), num_ticks)
cbar1.set_ticks(c_ticks)

# Set the fontweight and fontsize for color bar tick labels
for t in cbar1.ax.get_yticklabels():
    t.set_weight('normal')
    t.set_fontsize(26)  # Increase the font size for color bar tick labels

# Increase the size of numbers on axis and color bar
plt.xticks(fontsize=26)
plt.yticks(fontsize=26)

# Increase the tick size and width of the color bar
cbar1.ax.tick_params(axis='both', which='major', labelsize=30, width=3,  length=10)

#plt.savefig('Contour_Exact.pdf', dpi=500, bbox_inches="tight")
plt.savefig('contour_exact.jpeg', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator

# Assuming you have defined concatenated_tensor as a PyTorch tensor
# concatenated_tensor = torch.cat((tensor1, tensor2), dim=0)

# Convert concatenated_tensor to a NumPy array
concatenated_array = concatenated_tensor.numpy()

# Define custom color levels
x = np.linspace(-1, 1, concatenated_array.shape[1])  # Replace 0 and 1 with your actual x range
t = np.linspace(0, 1, concatenated_array.shape[0])  # Replace 0 and 1 with your actual t range
X, T = np.meshgrid(x, t1)

# Define custom color levels using the minimum and maximum from the NumPy array
c_levels = np.linspace(np.min(concatenated_array), np.max(concatenated_array), 400)

# Plot the contour with interpolated data
plt.figure(figsize=(20, 5))
plt.pcolormesh(T, X, concatenated_array, shading='auto', cmap='coolwarm')

# Set the fontweight for axis labels to regular (not bold)
plt.xlabel("$t$", fontsize=26)
plt.ylabel("$x$", fontsize=26)
plt.title("$u(x, t)$", fontsize=26)

# Set tick labels fontweight to regular (not bold) and increase font size
plt.tick_params(axis='both', which='major', labelsize=20, width=3, length=10)

# Set the fontweight for tick labels to regular (not bold)
for tick in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
    tick.set_weight('normal')

# Set the number of ticks for x-axis and y-axis to 5
num_ticks = 5
x_ticks = np.linspace(np.min(T), np.max(T), num_ticks)
y_ticks = np.linspace(np.min(X), np.max(X), num_ticks)

plt.gca().xaxis.set_major_locator(FixedLocator(x_ticks))
plt.gca().yaxis.set_major_locator(FixedLocator(y_ticks))

cbar1 = plt.colorbar()
# Set the number of ticks for the color bar with uniformly distributed numbers
num_ticks = 5
c_ticks = np.linspace(np.min(concatenated_array), np.max(concatenated_array), num_ticks)
cbar1.set_ticks(c_ticks)

# Set the fontweight and fontsize for color bar tick labels
for t in cbar1.ax.get_yticklabels():
    t.set_weight('normal')
    t.set_fontsize(26)  # Increase the font size for color bar tick labels

# Increase the size of numbers on axis and color bar
plt.xticks(fontsize=26)
plt.yticks(fontsize=26)

# Increase the tick size and width of the color bar
cbar1.ax.tick_params(axis='both', which='major', labelsize=30, width=3,  length=10)

# Add a dotted line at t = 0.8
plt.axvline(x=0.8, color='black', linestyle='dotted', linewidth=5)

#plt.savefig('Contour_LEM_20.pdf', dpi=500, bbox_inches="tight")
plt.savefig('contour_LEM_20.jpeg', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()
